In [ ]:
%load_ext lab_black

In [ ]:
import re
import nltk
from urllib.parse import urlparse
from yake import KeywordExtractor
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup as bsoup
from sqlitedict import SqliteDict as sqldict

In [ ]:
data_folder = "crawl"
responsedb = f"{data_folder}/responses.db"
kw_histdb = f"{data_folder}/kw_hist.db"

pstem = PorterStemmer()

# nltk.download("punkt")
nltk.download("stopwords")
stop_words = stopwords.words("english")

kwx = KeywordExtractor(lan="en", n=3, dedupLim=0.75, top=20, features=None)

In [ ]:
# Edit this to diminish influence of site name, etc.
more_stop_words = ["mike", "levin", "lpvg", "seo", "future-proof"]
stop_words = stop_words + more_stop_words

In [ ]:
with sqldict(responsedb) as db:
    with sqldict(kw_histdb) as db2:
        for i, url in enumerate(db):
            response = db[url]
            html = response.text
            text = bsoup(html, features="html.parser").get_text(
                separator=" ", strip=True
            )
            try:
                title = text.title.string.strip()
            except:
                title = None
            try:
                descr = text.find("meta", attrs={"name": "description"}).attrs[
                    "content"
                ]
            except:
                descr = None
            parts = urlparse(url)
            urlkws = re.split("\W+", parts.path)
            urlkws = " ".join([x for x in urlkws if x and not x.isnumeric()])
            text = " ".join([x for x in [title, urlkws, descr, text] if x])
            stripped = re.sub(r"[^ -~]", " ", text)
            stripped = re.sub(" +", " ", stripped)
            stemmed = pstem.stem(stripped)
            stemmed = " ".join([x for x in stemmed.split() if x not in stop_words])
            keywords = kwx.extract_keywords(stemmed)
            db2[url] = (keywords, stemmed)
            db2.commit()
print("Done")

In [ ]:
table = []
with sqldict(kw_histdb) as db:
    for url in db:
        data = db[url]
        histo, stemmed = data
        print(url)
        table.append(stemmed)
        for kw in histo:
            print(kw)
        print()